In [1]:
# merge_lists
#
# Merges the sea ice loss files using the "mean removed" and "bwfitler" methods into a combined file called "allvriles"
#
# Steven Cavallo
# December 2024

In [2]:
import numpy as np
import datetime as dt
import os, datetime
import matplotlib as mpl
from pathlib import Path
import gc

from mstats import *
import utilities_modules as um

print(mpl.__version__)

3.8.0


In [3]:
fpath_datain = '/Users/scavallo/Documents/data/seaice_loss/paper_data_directory/'
fpath_dataout = '/Users/scavallo/Documents/data/seaice_loss/paper_data_directory/'
months_filter = [1,12]

#months_filter = [3,5]
#months_filter = [6,8]
#months_filter = [9,11]
#months_filter = [12,2]

percentiles = [5,95] # For the data percentiles
#percentiles = [100,100]
dtUniqueCase = 1 # If set to 1, will not count back-to-back daily events as separate events

In [4]:
if ( (months_filter[0] == 1) & (months_filter[1] == 12) ):
    timeframe = 'annual'
if ( (months_filter[0] == 3) & (months_filter[1] == 5) ):
    timeframe = 'mam'
if ( (months_filter[0] == 6) & (months_filter[1] == 8) ):
    timeframe = 'jja'
if ( (months_filter[0] == 9) & (months_filter[1] == 11) ):
    timeframe = 'son'
if ( (months_filter[0] == 12) & (months_filter[1] == 2) ):
    timeframe = 'djf'

if dtUniqueCase == 0:
    dtname = ''    
else:
    dtname = '_dtUnique0' + str(dtUniqueCase)

percentilenum = percentiles[0] + (100-percentiles[1])
percstr = str(percentilenum) + 'percentile'

filtname = 'bwfilter'
fname_list1 = 'rapid_seaice_loss_events_int_withboth_' + timeframe + '_' + filtname + '_3d_' +percstr +  dtname +'.dat'
filtname = 'meanremoved'
fname_list2 = 'rapid_seaice_loss_events_int_withboth_' + timeframe + '_' + filtname + '_3d_' +percstr +  dtname +'.dat'
filtname = 'allvriles'
newfile = 'rapid_seaice_loss_events_int_withboth_' + timeframe + '_' + filtname + '_3d_' +percstr +  dtname +'.dat'


fpath_list1 = fpath_datain + fname_list1
fpath_list2 = fpath_datain + fname_list2
newfile_path = fpath_dataout + newfile

#event_fname_out = 'rapid_seaice_loss_events_int_withboth_' + timeframe + '_' + filtname + '_3d_' +percstr +  dtname +'.dat'
#print(event_fname_out)
#event_path_out = event_dir_out + event_fname_out

In [5]:
# read cases
print(fpath_list1)
aa = np.loadtxt(fpath_list1, skiprows=0)       
datelist_list1 = aa[:,0].astype(int)
dextent_in_list1 = aa[:,1]
dextent_in_nofilt_list1 = aa[:,2]

print(fpath_list2)
bb = np.loadtxt(fpath_list2, skiprows=0)       
datelist_list2 = bb[:,0].astype(int)
dextent_in_list2 = bb[:,1]
dextent_in_nofilt_list2 = bb[:,2]

test = list(set(datelist_list1).intersection(datelist_list2))


# These are the indices in datelist_list1 that also occur somewhere in datelist_list2
commondates = datelist_list1[np.nonzero(np.in1d(datelist_list1, datelist_list2))]
[inds] = np.nonzero(np.in1d(datelist_list1, datelist_list2))
#print(commondates)
#print(inds)
#print(datelist_list1[inds])

dumval = -9999
if 1 == 0:
    outfile = open(newfile_path,'w+')
    nvalues_file1 = len(datelist_list1)
    nvalues_file2 = len(datelist_list2)
    nvalues_total = nvalues_file1 + nvalues_file2
    nvalues_merged = nvalues_total - len(inds)
    datelist_merged = []
    values_nofilt_merged = []
    counter = 0
    indcounter = 0
    for ii in range(0,nvalues_total):
        #print(ii,nvalues_total)
        if ii < nvalues_file2:
            #print(datelist_list2[counter])
            outfile.write('%-10s %7.4f %7.4f\n' % (datelist_list2[counter], dumval, dextent_in_nofilt_list2[counter]))
            if ii == nvalues_file2-1:
                counter = -1
        else:  
            if indcounter < len(inds): 
                if counter != inds[indcounter]:
                    #print(datelist_list1[counter])
                    outfile.write('%-10s %7.4f %7.4f\n' % (datelist_list1[counter], dumval, dextent_in_nofilt_list1[counter]))
                else:
                    #print(counter,indcounter)
                    indcounter += 1
            else:
                outfile.write('%-10s %7.4f %7.4f\n' % (datelist_list1[counter], dumval, dextent_in_nofilt_list1[counter]))
        counter += 1


/Users/scavallo/Documents/data/seaice_loss/paper_data_directory/rapid_seaice_loss_events_int_withboth_annual_bwfilter_3d_100percentile.dat
/Users/scavallo/Documents/data/seaice_loss/paper_data_directory/rapid_seaice_loss_events_int_withboth_annual_meanremoved_3d_100percentile.dat


In [6]:
# No filter
if 1 == 0:
    outfile = open(newfile_path,'w+')
    nvalues_file1 = len(datelist_list1)
    nvalues_file2 = len(datelist_list2)
    nvalues_total = nvalues_file1 + nvalues_file2
    nvalues_merged = nvalues_total - len(inds)
    datelist_merged = []
    values_nofilt_merged = []
    counter = 0
    indcounter = 0
    for ii in range(0,nvalues_file1):
        #print(ii,indcounter)
        #if ( (ii <= len(inds)) & (ii == inds[indcounter])):
        #    indcounter+=1
        #else:
        outfile.write('%-10s %7.4f %7.4f\n' % (datelist_list1[counter], dumval, dextent_in_nofilt_list1[counter]))
        counter+=1
    counter = 0
    for ii in range(0,nvalues_file2):
        outfile.write('%-10s %7.4f %7.4f\n' % (datelist_list2[counter], dumval, dextent_in_nofilt_list2[counter]))
        counter+=1

In [7]:
# No repeated dates
if 1 == 1:
    outfile = open(newfile_path,'w+')
    nvalues_file1 = len(datelist_list1)
    nvalues_file2 = len(datelist_list2)
    nvalues_total = nvalues_file1 + nvalues_file2
    nvalues_merged = nvalues_total - len(inds)
    datelist_merged = []
    values_nofilt_merged = []
    counter = 0
    indcounter = 0

    datelist_list1_arr = np.array(datelist_list1).astype('i')
    datelist_list2_arr = np.array(datelist_list2).astype('i')
    
    
    datelist_combined = np.concatenate([datelist_list1_arr,datelist_list2_arr])
    datelist_out, indices = np.unique(datelist_combined , return_index=True)
    
    dextent_combined = np.concatenate([dextent_in_nofilt_list1,dextent_in_nofilt_list2])
    dextent_nofilt_combined = dextent_combined[indices]
    
    #datelist_union = np.union1d( datelist_list1_arr,datelist_list2_arr)
    
    counter = 0
    for ii in range(0,len(datelist_out)):
        outfile.write('%-10s %7.4f %7.4f\n' % (str(datelist_out[counter]), dumval, dextent_nofilt_combined[counter]))
        counter+=1